In [1]:
import pickle
import math
import pandas as pd
import os
from coir.data_loader import get_tasks, load_data_from_hf
from IPython.display import display, HTML
import ipywidgets as widgets
import html


In [2]:
csn_train_deepseek_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_deepseek_train_clean.csv"
csn_valid_deepseek_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_deepseek_valid_clean.csv"
csn_test_deepseek_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_deepseek_test_clean.csv"

csn_valid_granite_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_train_clean.csv"
csn_train_granite_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_valid_clean.csv"
csn_test_granite_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/postprocessing/output/CSN_granite_test_clean.csv"


tasks = get_tasks(tasks=["CodeSearchNet-python"])
corpus, queries, qrels = tasks["CodeSearchNet-python"]

# read them
df_train = pd.read_csv(csn_train_deepseek_path)
df_valid = pd.read_csv(csn_valid_deepseek_path)
df_test  = pd.read_csv(csn_test_deepseek_path)

# concatenate into one DataFrame
deepseek_df = pd.concat([df_train, df_valid, df_test], ignore_index=True)

df_train = pd.read_csv(csn_train_granite_path)
df_valid = pd.read_csv(csn_valid_granite_path)
df_test  = pd.read_csv(csn_test_granite_path)

granite_df = pd.concat([df_train, df_valid, df_test], ignore_index=True)

merged_df = pd.merge(
    deepseek_df,
    granite_df,
    on=["query_id", "corpus_id"],
    how="outer",
    suffixes=("_deepseek", "_granite")
)
merged_df.columns

in tasks 


fetched data from hf


Casting the dataset:   0%|          | 0/280310 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/280652 [00:00<?, ? examples/s]

data loader init


0it [00:00, ?it/s]

Index(['Unnamed: 0_deepseek', 'query_id', 'corpus_id', 'doc_deepseek',
       'code_deepseek', 'cleaned_code_deepseek', 'explanation_deepseek_1',
       'explanation_deepseek_2', 'explanation_deepseek_3',
       'explanation_deepseek_4', 'explanation_deepseek_5',
       'explanation_deepseek_1_cleaned', 'explanation_deepseek_2_cleaned',
       'explanation_deepseek_3_cleaned', 'explanation_deepseek_4_cleaned',
       'explanation_deepseek_5_cleaned', 'Unnamed: 0_granite', 'doc_granite',
       'code_granite', 'cleaned_code_granite', 'explanation_granite_1',
       'explanation_granite_2', 'explanation_granite_3',
       'explanation_granite_4', 'explanation_granite_5',
       'explanation_granite_1_cleaned', 'explanation_granite_2_cleaned',
       'explanation_granite_3_cleaned', 'explanation_granite_4_cleaned',
       'explanation_granite_5_cleaned'],
      dtype='object')

In [3]:
# Add Query Expnansion
qe_dataset = "/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_granite_temp_0.5_mmr_2.csv"
qe_df = pd.read_csv(qe_dataset)
qe_df.head()

,Unnamed: 0,id,original_query,mmr_rank,mmr_explanation
0,0,q1,python code to write bool value 1,1,Query:\npython code to write bool value 1\n\nE...
1,1,q1,python code to write bool value 1,2,Query:\npython code to write bool value 1\n\nE...
2,2,q1,python code to write bool value 1,3,Query:\npython code to write bool value 1\n\nE...
3,3,q1,python code to write bool value 1,4,Query:\npython code to write bool value 1\n\nE...
4,4,q1,python code to write bool value 1,5,Query:\npython code to write bool value 1\n\nE...


In [4]:
def mark_good_queries(df, k=10):
    """
    Given a retrieval DataFrame with columns:
      [query_id, retrieved_doc_id, score, ground_truth_relevance],
    returns a DataFrame with columns [query_id, is_good]
    where is_good is True if there's at least 1 relevant doc in top K.
    """
    df_sorted = df.sort_values(["query_id", "score"], ascending=[True, False])
    
    def top_k_has_relevant(subdf):
        topk = subdf.head(k)
        return int((topk["ground_truth_relevance"] == 1).any())

    query_good = df_sorted.groupby("query_id").apply(top_k_has_relevant).reset_index()
    query_good.columns = ["query_id", "is_good"]
    
    return query_good

def first_relevant_rank(df):
    df_sorted = df.sort_values(["query_id", "score"], ascending=[True, False])
    def get_rank(subdf):
        subdf = subdf.reset_index(drop=True)
        rel = subdf[subdf["ground_truth_relevance"] == 1]
        return rel.index[0] + 1 if not rel.empty else None
    rank_df = df_sorted.groupby("query_id").apply(get_rank).reset_index()
    rank_df.columns = ["query_id", "first_rel_rank"]
    return rank_df


Comparison of correct queries between methods

In [5]:
# LOAD RESULTS FOR METHOD 1
dataset_1 = "csn"
type_1 = "og"
method_1 = "baseline"
retrieval_1 = "dres"
encoder_1 = "intfloat/e5-base-v2"

if retrieval_1 == "bm25":
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_1}/{type_1}/{method_1}/{retrieval_1}/retrieval_evaluation.csv"
else:
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_1}/{type_1}/{method_1}/{retrieval_1}/{encoder_1}/retrieval_evaluation.csv"

first_df = pd.read_csv(results_path)

In [6]:
# LOAD RESULTS FOR METHOD 2
dataset_2 = "csn"
type_2 = "og"
method_2 = "granite1"
retrieval_2 = "dres"
encoder_2 = "intfloat/e5-base-v2"


if retrieval_2 == "bm25":
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_2}/{type_2}/{method_2}/{retrieval_2}/retrieval_evaluation.csv"
else:
    results_path = f"/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main/results/{dataset_2}/{type_2}/{method_2}/{retrieval_2}/{encoder_2}/retrieval_evaluation.csv"
second_df = pd.read_csv(results_path)

In [7]:
k = 10 # Hyper-param to select top k documents to classify a "good" retrieval
first_good = mark_good_queries(first_df, k=k)
second_good = mark_good_queries(second_df, k=k)

first_ranks = first_relevant_rank(first_df)
second_ranks = first_relevant_rank(second_df)

compare_df = first_good.merge(second_good, on="query_id", suffixes=(f"_{method_1}", f"_{method_2}"))
compare_df = compare_df.merge(first_ranks, on="query_id")
compare_df = compare_df.merge(second_ranks, on="query_id", suffixes=(f"_{method_1}", f"_{method_2}"))

compare_df.head()


,query_id,is_good_baseline,is_good_granite1,first_rel_rank_baseline,first_rel_rank_granite1
0,q265734,1,1,1.0,1.0
1,q265735,1,1,1.0,1.0
2,q265736,1,1,1.0,1.0
3,q265737,0,0,15.0,99.0
4,q265738,0,0,161.0,163.0


In [8]:
compare_df["query_text"] = compare_df["query_id"].map(queries)

bad_method_1 = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
print(f"Queries where {method_1} is good, but {method_2} is bad:")
print(bad_method_1)

bad_method_2 = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
print(f"Queries where {method_2} is good, but {method_1} is bad:")
print(bad_method_2)

# Queries both missed
both_missed = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
print("Queries both missed:")
print(both_missed)

# Queries both got
both_got = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
print("Queries both got:")
print(both_got)


Queries where baseline is good, but granite1 is bad:
      query_id  is_good_baseline  is_good_granite1  first_rel_rank_baseline  \
16     q265750                 1                 0                      6.0   
28     q265762                 1                 0                      4.0   
40     q265774                 1                 0                      1.0   
41     q265775                 1                 0                      1.0   
52     q265786                 1                 0                      3.0   
...        ...               ...               ...                      ...   
14881  q280615                 1                 0                      3.0   
14883  q280617                 1                 0                      4.0   
14885  q280619                 1                 0                      1.0   
14901  q280635                 1                 0                      7.0   
14912  q280646                 1                 0                      1.0   

In [9]:
good_method_1_count = compare_df[f"is_good_{method_1}"].sum()
good_method_2_count = compare_df[f"is_good_{method_2}"].sum()
bad_method_1_count = len(compare_df) - good_method_1_count
bad_method_2_count = len(compare_df) - good_method_2_count

print("Total queries:", len(compare_df))
print(f"{method_1.capitalize()} good queries:", good_method_1_count)
print(f"{method_1.capitalize()} bad queries:", bad_method_1_count)
print(f"{method_2.capitalize()} good queries:", good_method_2_count)
print(f"{method_2.capitalize()} bad queries:", bad_method_2_count)


# Queries where method_1 is good but method_2 is bad
bad_method_2_subset = compare_df[
    (compare_df[f"is_good_{method_1}"] == 1) &
    (compare_df[f"is_good_{method_2}"] == 0)
]
avg_rank_bad_method_2 = bad_method_2_subset[f"first_rel_rank_{method_2}"].mean()
print(f"\n{method_1.capitalize()} good but {method_2.capitalize()} bad queries count:", len(bad_method_2_subset))
#print(f"Average first relevant doc rank ({method_2.capitalize()}) for these queries:", avg_rank_bad_method_2)

# Queries where method_2 is good but method_1 is bad
bad_method_1_subset = compare_df[
    (compare_df[f"is_good_{method_1}"] == 0) &
    (compare_df[f"is_good_{method_2}"] == 1)
]
avg_rank_bad_method_1 = bad_method_1_subset[f"first_rel_rank_{method_1}"].mean()
print(f"\n{method_2.capitalize()} good but {method_1.capitalize()} bad queries count:", len(bad_method_1_subset))
#print(f"Average first relevant doc rank ({method_1.capitalize()}) for these queries:", avg_rank_bad_method_1)

# Queries both missed
print(f"\nQueries both missed: {len(both_missed)}")

# Queries both got
avg_rank_method_1_both = both_got[f"first_rel_rank_{method_1}"].mean()
avg_rank_method_2_both = both_got[f"first_rel_rank_{method_2}"].mean()
print(f"\nQueries both got: {len(both_got)}")
#print(f"Average first relevant doc rank ({method_1.capitalize()}) for these queries:", avg_rank_method_1_both)
#print(f"Average first relevant doc rank ({method_2.capitalize()}) for these queries:", avg_rank_method_2_both)

Total queries: 14918
Baseline good queries: 11332
Baseline bad queries: 3586
Granite1 good queries: 10837
Granite1 bad queries: 4081

Baseline good but Granite1 bad queries count: 1277

Granite1 good but Baseline bad queries count: 782

Queries both missed: 2804

Queries both got: 10055


In [10]:
def make_group_table(df, title):
    display_df = df[["query_id", "query_text"]].copy()
    html = f"<h3>{title} (n={len(display_df)})</h3>"
    html += display_df.to_html(index=False, escape=False)
    return html

html_out = ""
html_out += make_group_table(bad_method_1, f"Queries where {method_1} is good, but {method_2} is bad")
html_out += make_group_table(bad_method_2, f"Queries where {method_2} is good, but {method_1} is bad")
html_out += make_group_table(both_missed, "Queries both missed")
html_out += make_group_table(both_got, "Queries both got")

#display(HTML(html_out))

In [11]:
bad_method_1_merge = pd.merge(bad_method_1, merged_df, on="query_id", how="left")
bad_method_2_merge = pd.merge(bad_method_2, merged_df, on="query_id", how="left")
both_missed_merge = pd.merge(both_missed, merged_df, on="query_id", how="left")
both_got_merge = pd.merge(both_got, merged_df, on="query_id", how="left")

In [12]:
# Available columns for explanations and code.
for df in (merged_df, bad_method_1_merge, bad_method_2_merge, both_missed_merge, both_got_merge):
    # create a new 'code' column that’s a copy of 'code_deepseek'
    df['code'] = df['cleaned_code_deepseek']

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

available_explanation_columns = [
'explanation_granite_1_cleaned', 'explanation_granite_2_cleaned',
'explanation_granite_3_cleaned', 'explanation_granite_4_cleaned',
'explanation_granite_5_cleaned',
"explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
"explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
"explanation_deepseek_5_cleaned"
]

available_code_columns = [
    "code",
    "cleaned_code_deepseek",
    "explanation_deepseek_1_cleaned", "explanation_deepseek_2_cleaned", 
    "explanation_deepseek_3_cleaned", "explanation_deepseek_4_cleaned", 
    "explanation_deepseek_5_cleaned"
]

def make_html_block(text):
    """Wrap the given text in a <div> that preserves whitespace and left-aligns the text."""
    return f"<div style='white-space: pre-wrap; text-align: left; font-family: monospace;'>{text}</div>"

def show_page(df, code_col, expl_col, page=1, page_size=5):
    """
    Display a subset of rows from the DataFrame using the selected
    code column (for code) and explanation column (for explanation).
    The resulting table will have headers that exactly match the dropdown selections.
    """
    # Select the desired columns; keep original names.
    display_df = df[["query_id", "query_text", code_col, expl_col]].copy()
    
    # Apply HTML formatting to the code column.
    display_df[code_col] = display_df[code_col].apply(make_html_block)
    
    # Paginate.
    start = (page - 1) * page_size
    end = start + page_size
    sub_df = display_df.iloc[start:end]
    
    # Use the Styler to render HTML.
    styled = sub_df.style.format({code_col: lambda s: s})
    styled = styled.set_table_styles([
    {'selector': 'th', 
        'props': [('border', '1px solid black'),
                ('padding', '5px'),
                ('background-color', '#f0f0f0')]},
    {'selector': 'td', 
        'props': [('border', '1px solid black'),
                ('padding', '5px')]},
    # This applies alternating background colors to rows.
    {'selector': 'tr:nth-child(even)', 
        'props': [('background-color', '#f9f9f9')]}
])
    html = styled.to_html()
    display(HTML(html))

# Example interactive calls:
# (Replace the DataFrame variables below with your actual DataFrames,
# e.g. bad_method_1_merge, bad_method_2_merge, both_missed_merge, both_got_merge.)
print(f"Group: Queries where {method_1} is good, but {method_2} is bad")
widgets.interact(lambda page, code, expl: show_page(bad_method_1_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(bad_method_2_subset) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print(f"Group: Queries where {method_2} is good, but {method_1} is bad")
widgets.interact(lambda page, code, expl: show_page(bad_method_2_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(bad_method_1_subset) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print("Group: Queries both missed")
widgets.interact(lambda page, code, expl: show_page(both_missed_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(both_missed) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))

print("Group: Queries both got")
widgets.interact(lambda page, code, expl: show_page(both_got_merge, code, expl, page=page),
                 page=widgets.IntSlider(min=1, max=(len(both_got) // 5) + 1, step=1, value=1),
                 code=widgets.Dropdown(options=available_code_columns, value=available_code_columns[0], description="Code"),
                 expl=widgets.Dropdown(options=available_explanation_columns, value=available_explanation_columns[0], description="Explanation"))


Group: Queries where baseline is good, but granite1 is bad


interactive(children=(IntSlider(value=1, description='page', max=256, min=1), Dropdown(description='Code', opt…

Group: Queries where granite1 is good, but baseline is bad


interactive(children=(IntSlider(value=1, description='page', max=157, min=1), Dropdown(description='Code', opt…

Group: Queries both missed


interactive(children=(IntSlider(value=1, description='page', max=561, min=1), Dropdown(description='Code', opt…

Group: Queries both got


interactive(children=(IntSlider(value=1, description='page', max=2012, min=1), Dropdown(description='Code', op…

<function __main__.<lambda>(page, code, expl)>

In [14]:
query_disp_df1 = first_df.merge(
    merged_df,
    left_on=['retrieved_doc_id'],
    right_on=['corpus_id'],
    how='left'
)
query_disp_df2 = second_df.merge(
    merged_df,
    left_on=['retrieved_doc_id'],
    right_on=['corpus_id'],
    how='left'
)


METHOD 1 QUERY FILTERING

In [15]:
# ── 1) Prevent pandas from truncating long text ─
pd.set_option('display.max_colwidth', None)

# ── 2) Inject CSS for borders, wrapping, and monospace code ─
display(HTML('''
<style>
  table.dataframe {
    border-collapse: collapse;           /* collapse cell borders into single lines */
  }
  table.dataframe th,
  table.dataframe td {
    border: 1px solid #999;              /* 1px border around every cell */
    padding: 4px 8px;                    /* consistent padding */
    white-space: normal !important;      /* wrap long text */
    max-width: 250px;                    /* cap width */
    overflow-wrap: break-word;
    vertical-align: top;
  }
  /* monospace font & pre-wrap only for code cell content */
  table.dataframe td.code-cell div {
    font-family: monospace;
    white-space: pre-wrap;
  }
</style>
'''))

# ── helper to wrap code in a <div> for monospace & pre-wrap ─
def make_html_block(text):
    if pd.isna(text):
        return ""
    return f"<div style='white-space: pre-wrap; font-family: monospace;'>{text}</div>"

# ── 3) Columns you actually want to see ─
display_cols = [
    'retrieved_doc_id',
    'score',
    'ground_truth_relevance',
    'code',
    'explanation_granite_1_cleaned',
]

# ── 4) Build the widgets ─
qid_text     = widgets.Text(description='Query ID:', placeholder='e.g. q20105')
show_btn     = widgets.Button(description='Show')
page_slider  = widgets.IntSlider(value=1, min=1, max=1, step=1,
                                 description='Page:', continuous_update=False)
out          = widgets.Output()

# ── 5) App state and settings ─
state     = { 'df': pd.DataFrame(), 'query_text': '' }
PAGE_SIZE = 10

def refresh(page_num):
    """Show page `page_num` (1-based) in the `out` widget only."""
    out.clear_output(wait=True)
    df = state['df']
    if df.empty:
        with out:
            print("No results for that query.")
        return

    # Show the searched query above the table
    with out:
        display(HTML(f"<div style='font-weight:bold; margin-bottom:8px;'>🔍 Query: {state['query_text']}</div>"))

    start = (page_num - 1) * PAGE_SIZE
    end   = start + PAGE_SIZE
    total = len(df)

    # slice rows & columns, then format the code column
    sub = df.iloc[start:end][display_cols].copy()
    sub['code'] = sub['code'].apply(make_html_block)

    styled = (
        sub
        .style
        .set_td_classes(pd.DataFrame(
            [['code-cell' if col=='code' else '' for col in sub.columns]]
            * len(sub),
            columns=sub.columns
        ))
    )

    html = styled.to_html(escape=False)
    with out:
        display(HTML(html))
        print(f"Showing {start+1}–{min(end, total)} of {total}")

def on_show_clicked(_):
    qid = qid_text.value.strip()
    df_q = (
        query_disp_df1
        .query("query_id_x == @qid")
        .reset_index(drop=True)
    )
    # create 'code' alias from 'code_deepseek'
    if not df_q.empty and 'code_deepseek' in df_q.columns:
        df_q['code'] = df_q['code_deepseek']
    else:
        df_q['code'] = ""

    state['df'] = df_q
    # store actual query text
    state['query_text'] = df_q.loc[df_q['query_id_y'] == qid, 'doc_deepseek'].iloc[0] if not df_q.empty and 'doc_deepseek' in df_q.columns else ''

    # configure slider for how many pages we need
    n_pages = math.ceil(len(df_q) / PAGE_SIZE) if not df_q.empty else 1
    page_slider.max   = max(1, n_pages)
    page_slider.value = 1

    refresh(1)

def on_page_changed(change):
    if change['name']=='value' and change['new']!=change['old']:
        refresh(change['new'])

# ── 6) Wire up callbacks ─
show_btn.on_click(on_show_clicked)
page_slider.observe(on_page_changed, names='value')

# ── 7) Layout & display the UI once ─
ui = widgets.VBox([
    widgets.HBox([qid_text, show_btn]),
    page_slider,
    out
])
display(ui)


METHOD 2 QUERY FILTERING

In [16]:
# ── 1) Prevent pandas from truncating long text ─
pd.set_option('display.max_colwidth', None)

# ── 2) Inject CSS for borders, wrapping, and monospace code ─
display(HTML('''
<style>
  table.dataframe {
    border-collapse: collapse;           /* collapse cell borders into single lines */
  }
  table.dataframe th,
  table.dataframe td {
    border: 1px solid #999;              /* 1px border around every cell */
    padding: 4px 8px;                    /* consistent padding */
    white-space: normal !important;      /* wrap long text */
    max-width: 250px;                    /* cap width */
    overflow-wrap: break-word;
    vertical-align: top;
  }
  /* monospace font & pre-wrap only for code cell content */
  table.dataframe td.code-cell div {
    font-family: monospace;
    white-space: pre-wrap;
  }
</style>
'''))

# ── helper to wrap code in a <div> for monospace & pre-wrap ─
def make_html_block(text):
    if pd.isna(text):
        return ""
    return f"<div style='white-space: pre-wrap; font-family: monospace;'>{text}</div>"

# ── 3) Columns you actually want to see ─
display_cols = [
    'retrieved_doc_id',
    'score',
    'ground_truth_relevance',
    'code',
    'explanation_granite_1_cleaned',
]

# ── 4) Build the widgets ─
qid_text     = widgets.Text(description='Query ID:', placeholder='e.g. q20105')
show_btn     = widgets.Button(description='Show')
page_slider  = widgets.IntSlider(value=1, min=1, max=1, step=1,
                                 description='Page:', continuous_update=False)
out          = widgets.Output()

# ── 5) App state and settings ─
state     = { 'df': pd.DataFrame(), 'query_text': '' }
PAGE_SIZE = 10

def refresh(page_num):
    """Show page `page_num` (1-based) in the `out` widget only."""
    out.clear_output(wait=True)
    df = state['df']
    if df.empty:
        with out:
            print("No results for that query.")
        return

    # Show the searched query above the table
    with out:
        display(HTML(f"<div style='font-weight:bold; margin-bottom:8px;'>🔍 Query: {state['query_text']}</div>"))

    start = (page_num - 1) * PAGE_SIZE
    end   = start + PAGE_SIZE
    total = len(df)

    # slice rows & columns, then format the code column
    sub = df.iloc[start:end][display_cols].copy()
    sub['code'] = sub['code'].apply(make_html_block)

    styled = (
        sub
        .style
        .set_td_classes(pd.DataFrame(
            [['code-cell' if col=='code' else '' for col in sub.columns]]
            * len(sub),
            columns=sub.columns
        ))
    )

    html = styled.to_html(escape=False)
    with out:
        display(HTML(html))
        print(f"Showing {start+1}–{min(end, total)} of {total}")

def on_show_clicked(_):
    qid = qid_text.value.strip()
    df_q = (
        query_disp_df2
        .query("query_id_x == @qid")
        .reset_index(drop=True)
    )
    # create 'code' alias from 'code_deepseek'
    if not df_q.empty and 'code_deepseek' in df_q.columns:
        df_q['code'] = df_q['code_deepseek']
    else:
        df_q['code'] = ""

    state['df'] = df_q
    # store actual query text
    state['query_text'] = df_q.loc[df_q['query_id_y'] == qid, 'doc_deepseek'].iloc[0] if not df_q.empty and 'doc_deepseek' in df_q.columns else ''

    # configure slider for how many pages we need
    n_pages = math.ceil(len(df_q) / PAGE_SIZE) if not df_q.empty else 1
    page_slider.max   = max(1, n_pages)
    page_slider.value = 1

    refresh(1)

def on_page_changed(change):
    if change['name']=='value' and change['new']!=change['old']:
        refresh(change['new'])

# ── 6) Wire up callbacks ─
show_btn.on_click(on_show_clicked)
page_slider.observe(on_page_changed, names='value')

# ── 7) Layout & display the UI once ─
ui = widgets.VBox([
    widgets.HBox([qid_text, show_btn]),
    page_slider,
    out
])
display(ui)
